In [1]:
f = open('../spec/fixtures/finnish/finnish-task1-train').read().splitlines()[:10]

In [2]:
from collections import defaultdict

fk = defaultdict(dict)
inti = 0
for item in f:
    stuff = item.split('\t')
    fk[inti]['source'] = stuff[0]
    fk[inti]['inf'] = stuff[2]
    stuff2 = stuff[1].split(',')
    for thing in stuff2:
        thing2 = thing.split('=')
        fk[inti][thing2[0]] = thing2[1]
    inti += 1

In [3]:
import pandas
pd = pandas.DataFrame.from_dict(fk,orient='index')

# Ignore the statements below
nouns = pd[pd['pos']=='N'].dropna(axis=1)
nouns.groupby(['num','case']).count()
pd[pd['pos']=='N'].dropna(axis=1)
nouns[(nouns['num']=='PL') &(nouns['case']=='NOM') ].head(5)

,source,inf,pos,num,case
9,aaltopelti,aaltopellit,N,PL,NOM


In [16]:
rw3= pandas.DataFrame()
transM = pandas.DataFrame()
for i,item in pd.iterrows():
    charPairs = list()
    sourLen = len(item['source'])
    for j,chara in enumerate(item['source']):
        charPairs.append([i,chara,j+1,(sourLen - j)*-1])
        
    tempDF = pandas.DataFrame.from_records(charPairs,columns=['indi','chara','lpos','rpos'])
    tempTrans2 = pandas.merge(tempDF, tempDF,how='inner',on='indi')
    tempTrans = pandas.merge(tempDF, tempDF,how='outer',on='indi')
    print(tempTrans2)
    print(tempTrans)
    tempTrans = tempTrans[(tempTrans['chara_x'] != tempTrans['chara_y']) | (tempTrans['lpos_x'] != tempTrans['lpos_y'])  | (tempTrans['rpos_x'] != tempTrans['rpos_y']) ]
    
    try:
        transM = transM.append(tempTrans)
    except:
        transM = pandas.DataFrame(tempTrans)
        
    try:
        rw3 = rw3.append(tempDF)
    except NameError:
        rw3 = pandas.DataFrame(tempDF)
        print(rw3)
        
rw3 = rw3.reset_index(drop=True)
rw3
transM.shape

    indi chara_x  lpos_x  rpos_x chara_y  lpos_y  rpos_y
0      0       ä       1      -9       ä       1      -9
1      0       ä       1      -9       ä       2      -8
2      0       ä       1      -9       k       3      -7
3      0       ä       1      -9       k       4      -6
4      0       ä       1      -9       ö       5      -5
5      0       ä       1      -9       s       6      -4
6      0       ä       1      -9       t       7      -3
7      0       ä       1      -9       ä       8      -2
8      0       ä       1      -9       ä       9      -1
9      0       ä       2      -8       ä       1      -9
10     0       ä       2      -8       ä       2      -8
11     0       ä       2      -8       k       3      -7
12     0       ä       2      -8       k       4      -6
13     0       ä       2      -8       ö       5      -5
14     0       ä       2      -8       s       6      -4
15     0       ä       2      -8       t       7      -3
16     0       ä       2      -

    indi chara_x  lpos_x  rpos_x chara_y  lpos_y  rpos_y
0      4       a       1      -8       a       1      -8
1      4       a       1      -8       a       2      -7
2      4       a       1      -8       l       3      -6
3      4       a       1      -8       l       4      -5
4      4       a       1      -8       o       5      -4
5      4       a       1      -8       p       6      -3
6      4       a       1      -8       p       7      -2
7      4       a       1      -8       i       8      -1
8      4       a       2      -7       a       1      -8
9      4       a       2      -7       a       2      -7
10     4       a       2      -7       l       3      -6
11     4       a       2      -7       l       4      -5
12     4       a       2      -7       o       5      -4
13     4       a       2      -7       p       6      -3
14     4       a       2      -7       p       7      -2
15     4       a       2      -7       i       8      -1
16     4       l       3      -

    indi chara_x  lpos_x  rpos_x chara_y  lpos_y  rpos_y
0      9       a       1     -10       a       1     -10
1      9       a       1     -10       a       2      -9
2      9       a       1     -10       l       3      -8
3      9       a       1     -10       t       4      -7
4      9       a       1     -10       o       5      -6
5      9       a       1     -10       p       6      -5
6      9       a       1     -10       e       7      -4
7      9       a       1     -10       l       8      -3
8      9       a       1     -10       t       9      -2
9      9       a       1     -10       i      10      -1
10     9       a       2      -9       a       1     -10
11     9       a       2      -9       a       2      -9
12     9       a       2      -9       l       3      -8
13     9       a       2      -9       t       4      -7
14     9       a       2      -9       o       5      -6
15     9       a       2      -9       p       6      -5
16     9       a       2      -

(834, 7)

In [5]:
def lcs(SB,TB):
    S = SB
    T = TB
    m = len(S)
    n = len(T)
    
    
    globList = list()
    
    while True:
        lcs_set = set()
        lcsList = list()
        counter = [[0]*(n+1) for x in range(m+1)]
        longest = 0
        for i in range(m):
            for j in range(n):
                if S[i] == T[j]:
                    c = counter[i][j] + 1
                    counter[i+1][j+1] = c
                    if c > longest:
                        lcs_set = set()
                        longest = c
                        lcs_set.add(S[i-c+1:i+1])
                    elif c == longest:
                        lcs_set.add(S[i-c+1:i+1])
        
        lcsList = list(lcs_set)    
        globList.extend(lcsList)
        if len(lcs_set) == 0:
            break
        for item in lcsList:
            S = S.replace(item,'_',1)
            T = T.replace(item,'$',1)
            m = len(S)
            n = len(T)
    
    return [item for item in globList if len(item) > 1]    

In [15]:

rw4= pandas.DataFrame()
for i,item in pd.iterrows():
    charPairs = list()
    maxSeq = lcs(item['source'],item['inf'])

    str1,str2 = item['source'],item['inf']

    for pat in maxSeq:
        str1 = str1.replace(pat,'$'*len(pat),1)
        str2 = str2.replace(pat,'_'*len(pat),1)
    str1Rems = [stuff for stuff in str1.split('$') if len(stuff) > 0]
    str2Rems = [stuff for stuff in str2.split('_') if len(stuff) > 0]
    for  indx,chara in enumerate(str1):
        if chara != '$':
            if (len(str1)-indx)*-1 < 0:
                charPairs.append([i,'del',chara,indx+1,(len(str1)-indx)*-1])
            else:
                charPairs.append([i,'del',chara,indx+1,((len(str1)-indx)*-1)+1])
    for  indx,chara in enumerate(str2):
        if chara != '_':
            if (len(str1)-indx)*-1 < 0:
                charPairs.append([i,'del',chara,indx+1,(len(str1)-indx)*-1])
            else:
                charPairs.append([i,'del',chara,indx+1,((len(str1)-indx)*-1)+1])
    tempDF = pandas.DataFrame.from_records(charPairs,columns=['indi','opn','chara','lpos','rpos'])

    try:
        rw4 = rw4.append(tempDF)
    except NameError:
        rw4 = pandas.DataFrame(tempDF)
        print(rw4)

rw4 = rw4.reset_index(drop=True)
rw4

,indi,opn,chara,lpos,rpos
0,0,del,ä,9,-1
1,0,del,k,9,-1
2,0,del,ö,10,1
3,0,del,ö,11,2
4,0,del,n,12,3
5,1,del,ä,9,-1
6,1,del,m,9,-1
7,1,del,ä,10,1
8,1,del,i,11,2
9,1,del,s,12,3
